In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
import altair as alt
from altair_saver import save
import os
import re
from nltk.util import ngrams
from itertools import chain

import numpy as np
import warnings
warnings.simplefilter("ignore")

C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('Data_mesh_publications_cleaned.csv')
df_sample = df. copy()

In [3]:
# def generate_custom_token_pattern(ngram_min, ngram_max):
#     # Ensure no word repetition in n-grams
#     word_pattern = r'\b(\w+)\b(?=.*\b\1\b)'
    
#     # Combine word patterns for specified n-gram range
#     ngram_pattern = r'(?:' + word_pattern + r'\s){' + str(ngram_min - 1) + r',' + str(ngram_max - 1) + r'}' + word_pattern
    
#     return ngram_pattern

# token_pattern = generate_custom_token_pattern(1, 4)

In [4]:
# def extract_topics(text, tfidf_vectorizer, lda_model):
    
#     # TF-IDF Vectorization for the specific document
#     tfidf_data = tfidf_vectorizer.transform([text])
#     tfidf_values = tfidf_data.toarray()
#     feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
#     df_tfidf = pd.DataFrame(data=tfidf_values, columns=feature_names)

#     # Apply LDA
#     doc_topic_prob = lda_model.transform(df_tfidf)

#     # Get the most probable topic for the document
#     most_probable_topic = np.argmax(doc_topic_prob)

#     # Get the top words for the most probable topic with TF-IDF values
#     topic = lda_model.components_[most_probable_topic]
#     top_keywords_idx = topic.argsort()[:-10 - 1:-1]
#     top_keywords = [feature_names[i] for i in top_keywords_idx]
#     top_tfidf_values = [tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]
# #     top_tfidf_values = [tfidf_data[0, tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]


#     topic_words = list(zip(top_keywords, top_tfidf_values))

#     return most_probable_topic + 1, topic_words

# tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4), token_pattern=r'\b\w+\b')
# tfidf_data = tfidf_vectorizer.fit_transform(df['Content'])

# # Apply LDA
# lda = LatentDirichletAllocation(n_components=38, random_state=42)
# lda.fit(tfidf_data)


# # Assuming 'df' is your DataFrame with 'pdf_content' column
# df['topic'], df['topic_words'] = zip(*df.apply(lambda row: find_topic_words(row['Content']), axis=1))


In [5]:
import pandas as pd
from gensim import corpora

def find_topic_words(df, num_topics=36, num_words=10):
    """
    Find topic words for each content in the DataFrame using LDA.
    
    Parameters:
    - df: DataFrame with a 'Content' column containing preprocessed (tokenized) texts.
    - num_topics: Number of topics to extract.
    - num_words: Number of words to show for each topic.
    
    Returns:
    - topics_df: DataFrame with topics and their words for each document.
    """
    tokenize_content = df['Content'].apply(lambda x: word_tokenize(x.lower()))

    dictionary = corpora.Dictionary(tokenize_content)

    corpus = [dictionary.doc2bow(text) for text in tokenize_content]
    

    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    

    doc_topics = [lda_model.get_document_topics(bow) for bow in corpus]
    

    topics_df = pd.DataFrame()
    for n in range(len(df)):
        topic_num, prop_topic = sorted(doc_topics[n], key=lambda x: x[1], reverse=True)[0]  
        wp = lda_model.show_topic(topic_num, topn=num_words)
        topic_keywords = ", ".join([word for word, prop in wp])
        word_probs = [(word, round(prop, 4)) for word, prop in wp]  # Get topic words
        topics_df = topics_df.append(pd.Series([int(topic_num), topic_keywords, word_probs]), ignore_index=True)
    
    topics_df.columns = ['Dominant_Topic', 'Topic_Keywords','Word_Probabilities']
    return topics_df

In [6]:

topics_df = find_topic_words(df)
print(topics_df)

    Dominant_Topic                                     Topic_Keywords  \
0               21  data, localization, regulation, region, archit...   
1                6  data, product, port, output, example, concept,...   
2               15  data, product, quality, organization, concept,...   
3               12  data, architecture, warehouse, fabric, gartner...   
4               25  data, privacy, database, identi, quasi, er, di...   
5               23  data, product, service, analytical, microservi...   
6               29  data, architecture, integration, system, proce...   
7                3  data, product, smart, monitoring, architecture...   
8               19  data, ai, fabric, metadata, ml, architecture, ...   
9               32  data, product, dairy, farm, semantic, cowmesh,...   
10               9  data, product, gray, id, service, architecture...   
11              15  data, product, quality, organization, concept,...   
12              14  data, design, product, architec

In [7]:
merged_df = pd.concat([df_sample, topics_df], axis=1)


# Results with filename

In [8]:
result_data = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Filename = row['Filename']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data.append({'Filename': Filename, 'topic_word': word, 'prob': idf_value})

# Create the result DataFrame from the list
result_df = pd.DataFrame(result_data, columns=['Filename', 'topic_word', 'prob'])

# Print the result DataFrame
result_df

,Filename,topic_word,prob
0,Breaking Down Data Silos Data Mesh to Achieve ...,data,0.1560
1,Breaking Down Data Silos Data Mesh to Achieve ...,localization,0.0348
2,Breaking Down Data Silos Data Mesh to Achieve ...,regulation,0.0127
3,Breaking Down Data Silos Data Mesh to Achieve ...,region,0.0116
4,Breaking Down Data Silos Data Mesh to Achieve ...,architecture,0.0109
...,...,...,...
205,Utilization of Data Mesh Framework as a Part o...,different,0.0072
206,Utilization of Data Mesh Framework as a Part o...,product,0.0068
207,Utilization of Data Mesh Framework as a Part o...,new,0.0062
208,Utilization of Data Mesh Framework as a Part o...,system,0.0056


In [22]:
# Select the top terms for each PDF
top_terms= result_df.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms['prob'] = top_terms['prob'] + np.random.rand(top_terms.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= 0.1, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda= (heatmap + text).properties(width=1000)

# Select the top terms for each PDF



In [23]:
# save(chart_lda, "lda_idf_heatmap.html")
chart_lda

alt.LayerChart(...)

# Result with publication type

In [24]:
result_data_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publication_type.append({'Publication_type': Publication_type, 'topic_word': word, 'prob': idf_value})

# Create the result DataFrame from the list
result_df_publication_type = pd.DataFrame(result_data_publication_type, columns=['Publication_type', 'topic_word', 'prob'])

# Print the result DataFrame
result_df_publication_type

,Publication_type,topic_word,prob
0,conference,data,0.1560
1,conference,localization,0.0348
2,conference,regulation,0.0127
3,conference,region,0.0116
4,conference,architecture,0.0109
...,...,...,...
205,master thesis,different,0.0072
206,master thesis,product,0.0068
207,master thesis,new,0.0062
208,master thesis,system,0.0056


In [25]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_publication_type.groupby('Publication_type').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['prob'] = top_terms_df_publication_type['prob'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= 0.1, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publication_type = (heatmap + text).properties(width=2000)
chart_lda_publication_type

alt.LayerChart(...)

# Result with publisher

In [26]:
result_data_publisher = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Publisher = row['Publisher']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publisher.append({'Publisher': Publisher, 'topic_word': word, 'prob': idf_value})

# Create the result DataFrame from the list
result_df_publisher = pd.DataFrame(result_data_publisher)


result_df_publisher

,Publisher,topic_word,prob
0,IEEE,data,0.1560
1,IEEE,localization,0.0348
2,IEEE,regulation,0.0127
3,IEEE,region,0.0116
4,IEEE,architecture,0.0109
...,...,...,...
205,miscellaneous,different,0.0072
206,miscellaneous,product,0.0068
207,miscellaneous,new,0.0062
208,miscellaneous,system,0.0056


In [27]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['prob'] = top_terms_df_publisher['prob'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= 0.1, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publisher = (heatmap + text).properties(width=1500)
chart_lda_publisher

alt.LayerChart(...)

# Results with Year

In [16]:
result_data_date = []
for _, row in merged_df.iterrows():
    Year = row['Year']
    topic_words = row['Word_Probabilities']


    for word, prob in topic_words:
        result_data_date.append({'Year': Year, 'topic_word': word, 'prob': prob})


result_data_date = pd.DataFrame(result_data_date, columns=['Year', 'topic_word', 'prob'])

result_data_date

,Year,topic_word,prob
0,2023,data,0.1560
1,2023,localization,0.0348
2,2023,regulation,0.0127
3,2023,region,0.0116
4,2023,architecture,0.0109
...,...,...,...
205,2021,different,0.0072
206,2021,product,0.0068
207,2021,new,0.0062
208,2021,system,0.0056


In [19]:
# Select the top terms for each PDF
top_terms_df_dates = result_data_date.groupby('Year').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_dates['prob'] = top_terms_df_dates['prob'] + np.random.rand(top_terms_df_dates.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_dates).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Year:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Year"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= 0.1, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_date = (heatmap + text).properties(width=1500)
chart_lda_date

alt.LayerChart(...)

# USING ONLY TFIDF

In [30]:
def find_top_words_for_each_document(df, num_top_words=10):

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4), token_pattern=r'\b\w+\b')

    # Fit and transform the PDF content
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

    # Get feature names (words) from the TF-IDF vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame with TF-IDF values
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Now, for each document, find the top words based on TF-IDF scores
    top_words_for_each_document = []
    for i, row in enumerate(tfidf_df.iterrows()):
        _, document_tfidf_scores = row
        top_words_index = document_tfidf_scores.argsort()[-num_top_words:][::-1]
        top_words = [(feature_names[index], document_tfidf_scores[index]) for index in top_words_index]
        top_words_for_each_document.append(top_words)

    # Create a new DataFrame with 'pdf_content' and 'top_words'
    result_df = pd.DataFrame({'Filename': df['Filename'],'Year': df['Year'],'Publisher': df['Publisher'],'Publication_type': df['Publication_type'], 'Content': df['Content'], 'top_words': top_words_for_each_document})

    return result_df

In [31]:
# Call the function
result_df_tfidf = find_top_words_for_each_document(df)

# Display the result DataFrame
# result_df_tfidf['Publisher'] = df['Publisher']
# result_df_tfidf['Publication_type'] = df['Publication_type']
result_df_tfidf

,Filename,Year,Publisher,Publication_type,Content,top_words
0,Breaking Down Data Silos Data Mesh to Achieve ...,2023,IEEE,conference,abstract data localization law becoming make h...,"[(data, 0.5432637397245754), (localization, 0...."
1,Decentralized Data Governance as Part of a Dat...,2023,IEEE,conference,abstract data socio technical decentralized an...,"[(data, 0.6648525726760326), (product, 0.22029..."
2,Enterprise Data Strategy A Decentralized Data ...,2022,IEEE,conference,abstract enterprise experience exponential gro...,"[(data, 0.5323331850894721), (zone, 0.07964216..."
3,Finding Your Way Through the Jungle of Big Dat...,2021,IEEE,conference,abstract paper present systematic ofcommon ana...,"[(data, 0.45724825002307345), (archimate, 0.14..."
4,CoK A Survey of Privacy Challenges in Relation...,2022,Springer,conference,growing volume data appear multiple dis tribut...,"[(data, 0.28154726400989044), (privacy, 0.1866..."
5,Converging Data Mesh and Microservice Principl...,2023,Springer,conference,company invested driven design sup port data d...,"[(data, 0.385124224546545), (product, 0.094100..."
6,Data Integration Revitalized From Data Warehou...,2023,Springer,conference,year data integration architecture evolved fro...,"[(data, 0.4692560869296157), (ds, 0.1475454756..."
7,Data Mesh as Distributed Data Platform for Lar...,2023,Springer,conference,rapid increase data volume last decade promote...,"[(data, 0.558693298681318), (platform, 0.11061..."
8,Data Fabric and Data Mesh Approaches with AI,2023,Springer,book,look back data architecture developed response...,"[(data, 0.7347108390162084), (ai, 0.2302540431..."
9,Cow Mesh a data-mesh architecture to unify dai...,2023,ResearchGate,journal,dairy economically signi cant industry caters ...,"[(data, 0.48674364175718626), (dairy, 0.274347..."


# Results with filename

In [32]:
result_data_tfidf_filename = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Filename = row['Filename']
    top_words = row['top_words']

    # Iterate through each tuple in the list of top_words
    for word, tfidf in top_words:
        result_data_tfidf_filename.append({'Filename': Filename, 'topic_word': word, 'tfidf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_filename = pd.DataFrame(result_data_tfidf_filename)

result_df_tfidf_filename

,Filename,topic_word,tfidf_value
0,Breaking Down Data Silos Data Mesh to Achieve ...,data,0.543264
1,Breaking Down Data Silos Data Mesh to Achieve ...,localization,0.445187
2,Breaking Down Data Silos Data Mesh to Achieve ...,data localization,0.416301
3,Breaking Down Data Silos Data Mesh to Achieve ...,region,0.107965
4,Breaking Down Data Silos Data Mesh to Achieve ...,india,0.086793
...,...,...,...
205,Utilization of Data Mesh Framework as a Part o...,product,0.062669
206,Utilization of Data Mesh Framework as a Part o...,different,0.061351
207,Utilization of Data Mesh Framework as a Part o...,architecture,0.053360
208,Utilization of Data Mesh Framework as a Part o...,evans,0.053041


In [33]:
import pandas as pd

# Assuming result_df_3 is your DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)# Show all rows

# Display the entire DataFrame
print(result_df_tfidf_filename)


                                                                                         Filename  \
0        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
1        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
2        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
3        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
4        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
5        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
6        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
7        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
8        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Locali

In [34]:
# Select the top terms for each PDF
top_terms_df_tfidf = result_df_tfidf_filename.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf['tfidf_value'] = top_terms_df_tfidf['tfidf_value'] + np.random.rand(top_terms_df_tfidf.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_tfidf).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('tfidf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tfidf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.tfidf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf = (heatmap + text).properties(width=2000)


In [35]:
# save(chart_tfidf, 'tfidf_heatmap.html')
chart_tfidf

alt.LayerChart(...)

# Result with publisher

In [36]:
result_data_tfidf_publisher = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publisher = row['Publisher']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publisher.append({'Publisher': Publisher, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publisher = pd.DataFrame(result_data_tfidf_publisher)


result_df_tfidf_publisher

,Publisher,top_word,tf_idf_value
0,IEEE,data,0.543264
1,IEEE,localization,0.445187
2,IEEE,data localization,0.416301
3,IEEE,region,0.107965
4,IEEE,india,0.086793
5,IEEE,regulation,0.076302
6,IEEE,indian,0.054374
7,IEEE,organization,0.051427
8,IEEE,border,0.051397
9,IEEE,data aggregation,0.050045


In [37]:
result_df_tfidf_publisher.describe()

,tf_idf_value
count,210.000000
mean,0.147618
std,0.156410
min,0.039902
25%,0.064598
50%,0.084222
75%,0.139796
max,0.815885


In [38]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_tfidf_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['tf_idf_value'] = top_terms_df_publisher['tf_idf_value'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publisher = (heatmap + text).properties(width=1500)
chart_publisher

alt.LayerChart(...)

In [25]:
# save(chart_tfidf_publisher_2, 'tfidf_heatmap_publisher.html')

# Results with Publisher

In [39]:
result_data_tfidf_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publication_type.append({'Publication_type': Publication_type, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publication_type = pd.DataFrame(result_data_tfidf_publication_type)


result_df_tfidf_publication_type

,Publication_type,top_word,tf_idf_value
0,conference,data,0.543264
1,conference,localization,0.445187
2,conference,data localization,0.416301
3,conference,region,0.107965
4,conference,india,0.086793
5,conference,regulation,0.076302
6,conference,indian,0.054374
7,conference,organization,0.051427
8,conference,border,0.051397
9,conference,data aggregation,0.050045


In [40]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_tfidf_publication_type.groupby('Publication_type').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['tf_idf_value'] = top_terms_df_publication_type['tf_idf_value'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publication_type = (heatmap + text).properties(width=1500)
chart_publication_type

alt.LayerChart(...)

In [41]:
result_data_tfidf_date = []

In [43]:


# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Year = row['Year']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_date.append({'Year': Year, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_data_tfidf_date = pd.DataFrame(result_data_tfidf_date)


result_data_tfidf_date

,Year,top_word,tf_idf_value
0,2023,data,0.543264
1,2023,localization,0.445187
2,2023,data localization,0.416301
3,2023,region,0.107965
4,2023,india,0.086793
5,2023,regulation,0.076302
6,2023,indian,0.054374
7,2023,organization,0.051427
8,2023,border,0.051397
9,2023,data aggregation,0.050045


In [46]:
# Select the top terms for each PDF
top_terms_df_tfidf_dates = result_data_tfidf_date.groupby('Year').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf_dates['tf_idf_value'] = top_terms_df_tfidf_dates['tf_idf_value'] + np.random.rand(top_terms_df_tfidf_dates.shape[0]) * 0.0001

# Create a base chart
# Create a base chart
base = alt.Chart(top_terms_df_tfidf_dates).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Year:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Year"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_date_tfidf_type = (heatmap + text).properties(width=1500)
chart_date_tfidf_type

alt.LayerChart(...)